In [2]:
import konlpy
from konlpy.tag import Kkma, Komoran, Hannanum, Okt
from konlpy.utils import pprint
okt = Okt()
kkma = Kkma()

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

import re
from collections import Counter

import warnings
warnings.simplefilter("ignore")

from wordcloud import WordCloud
import matplotlib.pyplot as plt


from nltk.tokenize import word_tokenize
from nltk.tag import pos_tag

# 기초 불용어 처리
from nltk.corpus import stopwords

# 경고 뜨지 않게 설정
import warnings
warnings.filterwarnings('ignore')

# 형태소 벡터를 생성하기 위한 라이브러리
from sklearn.feature_extraction.text import CountVectorizer
# 형태소 벡터를 학습 벡터로 변환한다.
from sklearn.feature_extraction.text import TfidfTransformer

# 워드 클라우드를 위한 라이브러리
from collections import Counter
# import pytagcloud
from IPython.display import Image

# 저장
import pickle

import json
import ast

In [3]:
df = pd.read_excel(r"C:\Users\NT550009\Desktop\data\Rate.xlsx")

In [4]:
df.head()

,ProductNum,ID,Date,Grade,Text,Option,Size,FilteredText,Rate,FilteredLen
0,3569992365,날까,2023.04.08,4,쿠폰써서 2만원정도에 6벌이면 정말 싸요. 싼만큼 마감이 안좋은 부분도 있네요. 어...,"White 6-pack, 6, Medium",비공개,"['쿠폰', '써서', '정도', '벌이', '정말', '싸요', '싼만큼', '마...",2,18
1,3569992365,newr*****,2023.03.27,5,옷에 비침도 크게 없고 사이즈도 넉넉해서 좋습니다. 이번 여름까지 무난하게 잘 입을...,"XX-Large, 6, White 6-pack",비공개,"['비침도', '크게', '없고', '사이즈', '넉넉해서', '좋습니다', '여름...",2,10
2,3569992365,soch*****,2023.03.31,4,잘 알려진 브랜드입니다.면이 얇고 빨면 줄기도 합니다.올 여름 편하게 입으려고 구매...,"X-Large, 6, Charcoal Heather 6-pack",비공개,"['알려진', '브랜드', '입니다면이', '얇고', '줄기', '합니다', '여름...",2,13
3,3569992365,2045***************,2023.03.02,5,얇아서 비침이 있으며 꽤 부드러운 재질입니다.이너로 입기 괜찮을듯,"6, Large, White 6-pack",비공개,"['얇아서', '비침이', '있으며', '부드러운', '재질', '입니다', '입기...",1,8
4,3569992365,하이하이저요,2023.03.21,3,저렴하게 막 입기 좋네요생각보다 정사이즈인거 같아요 라지입는데 미듐샀더니가슴둘레가 ...,"Medium, 6, Navy 6-pack",비공개,"['저렴하게', '입기', '좋네요', '생각', '보다', '정사', '이즈', ...",2,20


In [5]:
zero_text_count = df.loc[df['Text'] == 0].shape[0]
print(zero_text_count)

1760


In [6]:
df.iloc[7125,5 ]

0

In [7]:
print(type(df.iloc[7125,5]))

<class 'int'>


In [8]:
len(df)-zero_text_count

5407

In [9]:
# 뭐 때문에 사이즈에 불만이 많았는지

# - 사이즈가 언급된 평가가 부정적인지?
# - 부정적인 평가에 사이즈 관련 언급이 있는지

In [10]:
# 1) Grade 와 Rate 의 상관관계
df.corr(method='pearson', min_periods=1)

,ProductNum,Grade,Rate,FilteredLen
ProductNum,1.000000,0.043403,0.022687,0.045276
Grade,0.043403,1.000000,0.192168,-0.095191
Rate,0.022687,0.192168,1.000000,0.210656
FilteredLen,0.045276,-0.095191,0.210656,1.000000


In [12]:
# df = df.drop('Unnamed: 0', axis=1)

In [13]:
df.head()

,ProductNum,ID,Date,Grade,Text,Option,Size,FilteredText,Rate,FilteredLen
0,3569992365,날까,2023.04.08,4,쿠폰써서 2만원정도에 6벌이면 정말 싸요. 싼만큼 마감이 안좋은 부분도 있네요. 어...,"White 6-pack, 6, Medium",비공개,"['쿠폰', '써서', '정도', '벌이', '정말', '싸요', '싼만큼', '마...",2,18
1,3569992365,newr*****,2023.03.27,5,옷에 비침도 크게 없고 사이즈도 넉넉해서 좋습니다. 이번 여름까지 무난하게 잘 입을...,"XX-Large, 6, White 6-pack",비공개,"['비침도', '크게', '없고', '사이즈', '넉넉해서', '좋습니다', '여름...",2,10
2,3569992365,soch*****,2023.03.31,4,잘 알려진 브랜드입니다.면이 얇고 빨면 줄기도 합니다.올 여름 편하게 입으려고 구매...,"X-Large, 6, Charcoal Heather 6-pack",비공개,"['알려진', '브랜드', '입니다면이', '얇고', '줄기', '합니다', '여름...",2,13
3,3569992365,2045***************,2023.03.02,5,얇아서 비침이 있으며 꽤 부드러운 재질입니다.이너로 입기 괜찮을듯,"6, Large, White 6-pack",비공개,"['얇아서', '비침이', '있으며', '부드러운', '재질', '입니다', '입기...",1,8
4,3569992365,하이하이저요,2023.03.21,3,저렴하게 막 입기 좋네요생각보다 정사이즈인거 같아요 라지입는데 미듐샀더니가슴둘레가 ...,"Medium, 6, Navy 6-pack",비공개,"['저렴하게', '입기', '좋네요', '생각', '보다', '정사', '이즈', ...",2,20


In [14]:
df.corr(method='pearson', min_periods=1)

,ProductNum,Grade,Rate,FilteredLen
ProductNum,1.000000,0.043403,0.022687,0.045276
Grade,0.043403,1.000000,0.192168,-0.095191
Rate,0.022687,0.192168,1.000000,0.210656
FilteredLen,0.045276,-0.095191,0.210656,1.000000


In [16]:
df = df.rename(columns = {'Unnamed':'Idx'})

In [ ]:
df = df.drop('Idx', axis=1)

In [ ]:
df.head()

In [17]:
df_re = pd.read_excel(r"C:\Users\NT550009\Desktop\data\Rate.xlsx")

In [18]:
df.head()

,ProductNum,ID,Date,Grade,Text,Option,Size,FilteredText,Rate,FilteredLen
0,3569992365,날까,2023.04.08,4,쿠폰써서 2만원정도에 6벌이면 정말 싸요. 싼만큼 마감이 안좋은 부분도 있네요. 어...,"White 6-pack, 6, Medium",비공개,"['쿠폰', '써서', '정도', '벌이', '정말', '싸요', '싼만큼', '마...",2,18
1,3569992365,newr*****,2023.03.27,5,옷에 비침도 크게 없고 사이즈도 넉넉해서 좋습니다. 이번 여름까지 무난하게 잘 입을...,"XX-Large, 6, White 6-pack",비공개,"['비침도', '크게', '없고', '사이즈', '넉넉해서', '좋습니다', '여름...",2,10
2,3569992365,soch*****,2023.03.31,4,잘 알려진 브랜드입니다.면이 얇고 빨면 줄기도 합니다.올 여름 편하게 입으려고 구매...,"X-Large, 6, Charcoal Heather 6-pack",비공개,"['알려진', '브랜드', '입니다면이', '얇고', '줄기', '합니다', '여름...",2,13
3,3569992365,2045***************,2023.03.02,5,얇아서 비침이 있으며 꽤 부드러운 재질입니다.이너로 입기 괜찮을듯,"6, Large, White 6-pack",비공개,"['얇아서', '비침이', '있으며', '부드러운', '재질', '입니다', '입기...",1,8
4,3569992365,하이하이저요,2023.03.21,3,저렴하게 막 입기 좋네요생각보다 정사이즈인거 같아요 라지입는데 미듐샀더니가슴둘레가 ...,"Medium, 6, Navy 6-pack",비공개,"['저렴하게', '입기', '좋네요', '생각', '보다', '정사', '이즈', ...",2,20


In [ ]:
######### 전반적인 상관관계 ###########

In [19]:
#  평점과 텍스트리뷰 긍부정 상관관계 
#'Grade', 'Rate' 컬럼 간 피어슨 상관계수 계산
corr = df[['Grade', 'Rate']].corr()
print(corr)

          Grade      Rate
Grade  1.000000  0.192168
Rate   0.192168  1.000000


In [20]:
# 평점과 리뷰길이 상관관계
# 'Grade' 와 'FilteredLen' 상관관계
corr = df[['Grade', 'FilteredLen']].corr()
print(corr)

                Grade  FilteredLen
Grade        1.000000    -0.095191
FilteredLen -0.095191     1.000000


In [21]:
# 'Rate' 와 'FilteredLen' 상관관계
corr = df[['Rate', 'FilteredLen']].corr()
print(corr)

                 Rate  FilteredLen
Rate         1.000000     0.210656
FilteredLen  0.210656     1.000000


In [ ]:
# df.iloc[row, col]
# 3 Grade	4 Text	5 Option	 6 Size	 7 FilteredText	 8 Rate	 9FilteredLen

df.iloc[1, 3]

In [ ]:
df.tail()

In [ ]:
print(len(df))

In [22]:
size_exist = [1, 17, 18, 20, 22, 24, 25, 27, 33, 36, 38, 42, 43, 47, 48, 50, 51, 55, 60, 61, 62, 63, 64, 65, 67, 68, 70, 71, 72, 73, 75, 78, 79, 82, 83, 86, 91, 99, 101, 104, 115, 116, 118, 119, 120, 122, 151, 156, 158, 160, 169, 177, 180, 279, 282, 293, 297, 301, 302, 308, 309, 310, 311, 314, 315, 318, 319, 320, 321, 326, 328, 332, 333, 339, 345, 348, 349, 350, 353, 357, 358, 359, 360, 365, 366, 367, 368, 369, 370, 371, 372, 375, 376, 377, 382, 383, 384, 385, 388, 389, 390, 394, 395, 397, 398, 400, 402, 408, 411, 412, 413, 416, 417, 419, 421, 425, 430, 431, 432, 439, 443, 444, 445, 446, 457, 462, 474, 475, 477, 486, 487, 490, 493, 494, 507, 508, 528, 548, 551, 552, 555, 558, 572, 573, 575, 582, 583, 588, 591, 738, 741, 743, 749, 751, 752, 796, 797, 798, 799, 800, 802, 804, 806, 807, 817, 819, 820, 827, 832, 835, 839, 840, 841, 842, 843, 845, 847, 848, 853, 858, 859, 867, 869, 872, 873, 879, 880, 890, 896, 899, 911, 915, 922, 926, 927, 944, 963, 965, 971, 975, 977, 980, 989, 992, 996, 1000, 1002, 1021, 1136, 1138, 1155, 1164, 1181, 1182, 1185, 1186, 1187, 1192, 1196, 1197, 1198, 1200, 1205, 1207, 1211, 1213, 1216, 1223, 1229, 1230, 1231, 1235, 1241, 1261, 1271, 1274, 1275, 1277, 1278, 1280, 1281, 1292, 1293, 1296, 1297, 1299, 1300, 1303, 1310, 1316, 1317, 1318, 1319, 1320, 1323, 1324, 1325, 1332, 1335, 1346, 1347, 1350, 1356, 1361, 1367, 1369, 1372, 1374, 1376, 1378, 1379, 1380, 1392, 1397, 1408, 1409, 1412, 1413, 1414, 1419, 1421, 1425, 1434, 1456, 1466, 1468, 1480, 1486, 1501, 1504, 1607, 1621, 1635, 1643, 1644, 1647, 1648, 1651, 1654, 1655, 1656, 1658, 1668, 1673, 1675, 1677, 1678, 1679, 1681, 1682, 1687, 1689, 1691, 1694, 1700, 1705, 1708, 1713, 1733, 1735, 1737, 1738, 1742, 1796, 1798, 1799, 1810, 1828, 1830, 1835, 1836, 1837, 1838, 1840, 1841, 1850, 1858, 1868, 1869, 1870, 1872, 1873, 1876, 1880, 1882, 1884, 1887, 1890, 1893, 1894, 1895, 1897, 1902, 1904, 1905, 1909, 1910, 1911, 1914, 1916, 1917, 1919, 1922, 1923, 1924, 1927, 1928, 1937, 1938, 1940, 1941, 1948, 1954, 1960, 1963, 1970, 1972, 1975, 1976, 1978, 1979, 1993, 1998, 2009, 2015, 2101, 2110, 2116, 2117, 2122, 2125, 2128, 2129, 2151, 2154, 2160, 2169, 2172, 2173, 2175, 2180, 2181, 2186, 2189, 2191, 2193, 2194, 2195, 2202, 2203, 2208, 2212, 2213, 2214, 2215, 2218, 2219, 2221, 2223, 2226, 2228, 2231, 2232, 2233, 2236, 2242, 2243, 2244, 2250, 2258, 2263, 2269, 2285, 2293, 2299, 2323, 2338, 2390, 2406, 2408, 2409, 2417, 2419, 2421, 2424, 2427, 2430, 2431, 2432, 2434, 2436, 2439, 2440, 2441, 2443, 2445, 2446, 2447, 2448, 2449, 2454, 2455, 2458, 2462, 2465, 2466, 2471, 2474, 2479, 2481, 2482, 2483, 2484, 2488, 2489, 2490, 2491, 2492, 2498, 2506, 2507, 2512, 2513, 2517, 2522, 2523, 2525, 2529, 2534, 2541, 2551, 2555, 2557, 2563, 2564, 2570, 2572, 2574, 2576, 2582, 2584, 2593, 2595, 2596, 2603, 2605, 2609, 2614, 2618, 2633, 2648, 2664, 2665, 2670, 2759, 2769, 2772, 2773, 2775, 2791, 2794, 2795, 2796, 2800, 2801, 2810, 2812, 2820, 2822, 2823, 2824, 2826, 2827, 2833, 2838, 2844, 2846, 2876, 2882, 2884, 2900, 2901, 2904, 2905, 2906, 2907, 2908, 2910, 2913, 2914, 2918, 2920, 2921, 2922, 2925, 2926, 2928, 2931, 2933, 2935, 2936, 2937, 2945, 2946, 2958, 2959, 2960, 2962, 2963, 2965, 2967, 2969, 2971, 2972, 2973, 2974, 2977, 2978, 2979, 2984, 2985, 2988, 2989, 2991, 2992, 2994, 2995, 2996, 3001, 3002, 3003, 3004, 3006, 3007, 3008, 3010, 3023, 3041, 3044, 3045, 3046, 3058, 3104, 3105, 3106, 3110, 3114, 3115, 3125, 3127, 3131, 3135, 3138, 3140, 3147, 3148, 3149, 3151, 3152, 3153, 3154, 3157, 3158, 3160, 3166, 3168, 3169, 3170, 3176, 3179, 3181, 3182, 3194, 3196, 3203, 3205, 3240, 3242, 3243, 3244, 3245, 3250, 3252, 3253, 3258, 3259, 3261, 3262, 3267, 3268, 3271, 3272, 3274, 3275, 3276, 3277, 3278, 3280, 3286, 3287, 3288, 3289, 3293, 3294, 3295, 3298, 3302, 3303, 3307, 3329, 3330, 3335, 3339, 3341, 3352, 3359, 3360, 3363, 3368, 3369, 3370, 3371, 3372, 3389, 3390, 3392, 3393, 3394, 3403, 3414, 3416, 3422, 3423, 3424, 3425, 3426, 3428, 3434, 3435, 3436, 3439, 3440, 3443, 3445, 3446, 3448, 3449, 3455, 3461, 3462, 3463, 3464, 3467, 3468, 3469, 3470, 3472, 3479, 3481, 3483, 3484, 3485, 3486, 3488, 3489, 3490, 3494, 3505, 3511, 3515, 3520, 3578, 3591, 3592, 3593, 3594, 3602, 3603, 3604, 3607, 3608, 3623, 3628, 3634, 3635, 3636, 3637, 3640, 3641, 3644, 3649, 3650, 3651, 3652, 3653, 3668, 3669, 3675, 3677, 3678, 3679, 3680, 3681, 3682, 3683, 3684, 3686, 3687, 3688, 3689, 3692, 3693, 3696, 3705, 3706, 3746, 3748, 3749, 3751, 3753, 3755, 3756, 3757, 3758, 3760, 3763, 3766, 3770, 3771, 3772, 3773, 3774, 3775, 3776, 3777, 3779, 3783, 3784, 3786, 3791, 3793, 3799, 3800, 3804, 3805, 3810, 3829, 3838, 3841, 3842, 3843, 3850, 3853, 3856, 3873, 3879, 3880, 3881, 3882, 3883, 3884, 3885, 3887, 3888, 3889, 3917, 3918, 3919, 3920, 3927, 3930, 3933, 3934, 3935, 3936, 3938, 3939, 3941, 3944, 3948, 3951, 3955, 3956, 3960, 3962, 3963, 3965, 3967, 3968, 3969, 3970, 3971, 3973, 3975, 3978, 3990, 3998, 4050, 4067, 4071, 4075, 4077, 4080, 4081, 4083, 4084, 4086, 4087, 4098, 4099, 4102, 4103, 4105, 4106, 4109, 4116, 4122, 4123, 4124, 4125, 4126, 4129, 4130, 4131, 4138, 4141, 4152, 4153, 4156, 4162, 4167, 4173, 4175, 4178, 4180, 4182, 4184, 4185, 4186, 4198, 4203, 4214, 4215, 4218, 4219, 4220, 4225, 4227, 4231, 4240, 4262, 4272, 4274, 4286, 4292, 4307, 4310, 4413, 4427, 4438, 4445, 4448, 4454, 4461, 4462, 4468, 4469, 4470, 4471, 4474, 4475, 4478, 4479, 4480, 4481, 4486, 4488, 4492, 4493, 4499, 4505, 4508, 4509, 4510, 4513, 4517, 4518, 4519, 4520, 4525, 4526, 4527, 4528, 4529, 4530, 4531, 4532, 4535, 4536, 4537, 4542, 4543, 4544, 4545, 4548, 4549, 4550, 4554, 4555, 4557, 4558, 4560, 4562, 4568, 4571, 4572, 4573, 4576, 4577, 4579, 4581, 4585, 4590, 4591, 4592, 4599, 4603, 4604, 4605, 4606, 4617, 4622, 4634, 4635, 4637, 4646, 4647, 4650, 4653, 4654, 4667, 4668, 4688, 4708, 4711, 4712, 4715, 4718, 4732, 4733, 4735, 4742, 4743, 4748, 4751, 4898, 4901, 4903, 4909, 4911, 4912, 4964, 4977, 4979, 4980, 4981, 4982, 4983, 4984, 4985, 4986, 4987, 4989, 4993, 4996, 4998, 4999, 5000, 5001, 5003, 5005, 5009, 5014, 5015, 5017, 5018, 5019, 5024, 5031, 5038, 5044, 5045, 5081, 5095, 5098, 5100, 5102, 5104, 5106, 5109, 5111, 5112, 5113, 5115, 5116, 5117, 5118, 5119, 5120, 5122, 5123, 5126, 5127, 5128, 5129, 5130, 5133, 5134, 5137, 5142, 5144, 5145, 5168, 5178, 5181, 5182, 5186, 5187, 5188, 5189, 5191, 5192, 5193, 5196, 5197, 5198, 5200, 5202, 5203, 5204, 5206, 5207, 5208, 5217, 5220, 5221, 5222, 5227, 5231, 5232, 5234, 5236, 5238, 5239, 5240, 5242, 5243, 5248, 5252, 5253, 5255, 5256, 5262, 5278, 5283, 5343, 5348, 5349, 5354, 5355, 5360, 5362, 5364, 5365, 5370, 5372, 5373, 5374, 5377, 5399, 5412, 5413, 5414, 5415, 5416, 5417, 5419, 5421, 5423, 5424, 5427, 5428, 5433, 5438, 5441, 5444, 5446, 5447, 5450, 5451, 5454, 5455, 5457, 5460, 5484, 5486, 5489, 5491, 5494, 5495, 5497, 5502, 5503, 5504, 5506, 5512, 5514, 5515, 5517, 5518, 5521, 5524, 5525, 5529, 5531, 5541, 5572, 5573, 5596, 5597, 5598, 5599, 5600, 5601, 5602, 5604, 5608, 5610, 5611, 5613, 5614, 5616, 5622, 5623, 5642, 5652, 5653, 5654, 5655, 5656, 5657, 5658, 5660, 5661, 5663, 5664, 5665, 5668, 5670, 5671, 5680, 5681, 5682, 5683, 5684, 5690, 5691, 5692, 5693, 5696, 5697, 5700, 5701, 5702, 5703, 5708, 5710, 5714, 5715, 5721, 5727, 5730, 5731, 5732, 5735, 5739, 5740, 5741, 5742, 5747, 5748, 5749, 5750, 5751, 5752, 5753, 5754, 5757, 5758, 5759, 5764, 5765, 5766, 5767, 5770, 5771, 5772, 5776, 5777, 5779, 5780, 5782, 5784, 5790, 5793, 5794, 5795, 5798, 5799, 5801, 5803, 5807, 5812, 5813, 5814, 5821, 5825, 5826, 5827, 5828, 5839, 5844, 5856, 5857, 5859, 5868, 5869, 5872, 5875, 5876, 5889, 5890, 5910, 5930, 5933, 5934, 5937, 5940, 5954, 5955, 5957, 5964, 5965, 5970, 5973, 6120, 6123, 6125, 6131, 6133, 6134, 6178, 6179, 6192, 6193, 6194, 6195, 6198, 6199, 6200, 6203, 6207, 6209, 6210, 6212, 6213, 6215, 6222, 6223, 6228, 6236, 6239, 6240, 6242, 6243, 6244, 6245, 6257, 6258, 6299, 6304, 6305, 6306, 6307, 6309, 6311, 6317, 6318, 6319, 6326, 6343, 6353, 6356, 6382, 6383, 6384, 6389, 6393, 6394, 6395, 6396, 6402, 6404, 6408, 6414, 6416, 6417, 6418, 6420, 6422, 6425, 6428, 6437, 6438, 6439, 6440, 6442, 6445, 6447, 6449, 6466, 6467, 6468, 6471, 6475, 6476, 6480, 6502, 6503, 6507, 6513, 6514, 6523, 6524, 6527, 6529, 6535, 6539, 6541, 6556, 6557, 6559, 6561, 6564, 6565, 6566, 6570, 6588, 6595, 6605, 6609, 6621, 6622, 6628, 6633, 6647, 6650, 6651, 6663, 6666, 6667, 6669, 6672, 6673, 6675, 6677, 6680, 6686, 6689, 6697, 6698, 6699, 6700, 6701, 6702, 6703, 6705, 6706, 6708, 6709, 6717, 6721, 6722, 6723, 6724, 6725, 6737, 6753, 6754, 6756, 6759, 6761, 6777, 6778, 6786, 6789, 6791, 6797, 6801, 6812, 6813, 6816, 6823, 6833, 6843, 6845, 6852, 6854, 6864, 6865, 6866, 6867, 6868, 6869, 6870, 6872, 6873, 6889, 6890, 6893, 6908, 6912, 6915, 6916, 6917, 6919, 6920, 6922, 6923, 6938, 6942, 6943, 6944, 6945, 6964, 6977, 6978, 6981, 6985, 6989, 6994, 6995, 6996, 6998, 7000, 7003, 7004, 7010, 7012, 7014, 7015, 7017, 7019, 7022, 7023, 7027, 7028, 7030, 7032, 7033, 7036, 7037, 7038, 7039, 7040, 7042, 7043, 7044, 7050, 7052, 7053, 7054, 7056, 7058, 7060, 7063, 7065, 7066, 7075, 7083, 7085, 7143, 7147, 7151, 7160, 7165]
price_exist = [21, 32, 42, 44, 47, 48, 49, 55, 60, 62, 75, 78, 84, 85, 95, 96, 97, 102, 104, 108, 124, 129, 132, 136, 140, 150, 154, 157, 160, 265, 266, 267, 301, 309, 310, 312, 313, 319, 321, 333, 338, 356, 361, 365, 367, 369, 370, 372, 379, 385, 389, 392, 395, 406, 408, 411, 416, 433, 436, 441, 444, 447, 459, 463, 481, 482, 487, 493, 494, 502, 503, 532, 556, 560, 568, 570, 586, 742, 747, 760, 761, 769, 770, 797, 799, 803, 807, 808, 811, 813, 816, 822, 827, 831, 835, 839, 851, 858, 864, 866, 881, 885, 891, 909, 914, 925, 935, 936, 943, 949, 966, 988, 993, 1006, 1121, 1123, 1160, 1172, 1178, 1179, 1183, 1185, 1186, 1187, 1188, 1191, 1194, 1203, 1204, 1205, 1213, 1220, 1221, 1237, 1261, 1276, 1296, 1299, 1306, 1308, 1309, 1314, 1322, 1330, 1331, 1343, 1349, 1362, 1364, 1369, 1376, 1379, 1387, 1388, 1394, 1399, 1415, 1417, 1429, 1453, 1469, 1475, 1476, 1477, 1491, 1492, 1509, 1601, 1606, 1631, 1632, 1633, 1634, 1646, 1650, 1651, 1655, 1657, 1659, 1661, 1666, 1667, 1672, 1675, 1680, 1683, 1687, 1692, 1705, 1709, 1713, 1715, 1743, 1746, 1824, 1827, 1833, 1837, 1839, 1852, 1854, 1867, 1876, 1880, 1881, 1891, 1892, 1894, 1901, 1904, 1907, 1908, 1918, 1925, 1929, 1931, 1935, 1964, 1968, 1970, 1988, 1998, 2005, 2007, 2008, 2021, 2103, 2109, 2111, 2112, 2113, 2118, 2119, 2123, 2136, 2137, 2145, 2147, 2173, 2177, 2178, 2182, 2189, 2196, 2201, 2206, 2211, 2214, 2222, 2224, 2225, 2227, 2229, 2234, 2237, 2240, 2246, 2276, 2288, 2290, 2298, 2413, 2414, 2415, 2418, 2420, 2428, 2433, 2450, 2461, 2464, 2468, 2480, 2485, 2497, 2499, 2502, 2508, 2510, 2512, 2515, 2518, 2538, 2552, 2560, 2561, 2571, 2575, 2581, 2613, 2621, 2642, 2646, 2652, 2662, 2663, 2767, 2789, 2792, 2808, 2809, 2819, 2826, 2831, 2840, 2842, 2843, 2853, 2880, 2883, 2884, 2889, 2902, 2904, 2905, 2919, 2923, 2927, 2928, 2930, 2931, 2943, 2954, 2960, 2964, 2973, 2974, 2983, 2986, 2995, 3006, 3015, 3018, 3020, 3048, 3055, 3135, 3145, 3146, 3151, 3158, 3161, 3162, 3186, 3187, 3188, 3193, 3208, 3211, 3245, 3279, 3294, 3310, 3342, 3360, 3404, 3428, 3430, 3448, 3452, 3457, 3465, 3466, 3473, 3480, 3482, 3501, 3585, 3589, 3590, 3595, 3605, 3619, 3623, 3641, 3682, 3697, 3740, 3743, 3746, 3751, 3752, 3769, 3845, 3852, 3879, 3896, 3900, 3918, 3924, 3925, 3946, 3961, 3968, 3987, 4005, 4082, 4102, 4105, 4112, 4114, 4115, 4120, 4128, 4136, 4137, 4149, 4155, 4168, 4170, 4175, 4182, 4185, 4193, 4194, 4200, 4205, 4221, 4223, 4235, 4259, 4275, 4281, 4282, 4283, 4297, 4298, 4315, 4407, 4412, 4461, 4469, 4470, 4472, 4473, 4479, 4481, 4493, 4498, 4516, 4521, 4525, 4527, 4529, 4530, 4532, 4539, 4545, 4549, 4552, 4555, 4566, 4568, 4571, 4576, 4593, 4596, 4601, 4604, 4607, 4619, 4623, 4641, 4642, 4647, 4653, 4654, 4662, 4663, 4692, 4716, 4720, 4728, 4730, 4746, 4902, 4907, 4920, 4921, 4929, 4930, 4983, 4988, 4989, 4991, 4999, 5001, 5005, 5011, 5014, 5023, 5031, 5032, 5124, 5125, 5148, 5176, 5179, 5190, 5197, 5198, 5200, 5205, 5213, 5218, 5280, 5281, 5288, 5289, 5364, 5369, 5410, 5426, 5449, 5451, 5495, 5501, 5504, 5505, 5514, 5517, 5518, 5523, 5526, 5582, 5591, 5598, 5602, 5604, 5605, 5616, 5652, 5683, 5691, 5692, 5694, 5695, 5701, 5703, 5715, 5720, 5738, 5743, 5747, 5749, 5751, 5752, 5754, 5761, 5767, 5771, 5774, 5777, 5788, 5790, 5793, 5798, 5815, 5818, 5823, 5826, 5829, 5841, 5845, 5863, 5864, 5869, 5875, 5876, 5884, 5885, 5914, 5938, 5942, 5950, 5952, 5968, 6124, 6129, 6142, 6143, 6151, 6152, 6178, 6202, 6206, 6304, 6313, 6315, 6326, 6347, 6386, 6400, 6404, 6417, 6419, 6421, 6427, 6430, 6437, 6445, 6451, 6467, 6483, 6523, 6529, 6546, 6564, 6619, 6620, 6666, 6670, 6689, 6691, 6696, 6697, 6709, 6715, 6720, 6728, 6779, 6789, 6826, 6830, 6831, 6843, 6846, 6860, 6864, 6892, 6913, 6933, 6942, 6943, 6949, 6972, 6977, 6995, 7008, 7011, 7020, 7026, 7034, 7044, 7163]
shipment_exist = [16, 31, 36, 41, 46, 52, 74, 88, 127, 133, 137, 143, 145, 148, 152, 167, 172, 173, 184, 186, 277, 284, 297, 298, 306, 311, 316, 320, 354, 372, 385, 390, 392, 393, 402, 450, 451, 452, 453, 454, 459, 463, 470, 478, 480, 497, 507, 515, 516, 517, 523, 533, 554, 565, 580, 581, 586, 589, 605, 607, 618, 731, 781, 796, 797, 809, 811, 812, 815, 831, 832, 838, 863, 871, 880, 887, 909, 910, 912, 920, 921, 922, 923, 924, 927, 929, 940, 943, 965, 985, 1010, 1011, 1018, 1029, 1124, 1125, 1126, 1138, 1146, 1165, 1191, 1194, 1205, 1215, 1220, 1224, 1227, 1229, 1230, 1231, 1234, 1296, 1309, 1317, 1326, 1330, 1349, 1351, 1352, 1375, 1377, 1388, 1399, 1400, 1401, 1402, 1403, 1404, 1421, 1427, 1446, 1491, 1492, 1507, 1611, 1615, 1616, 1617, 1622, 1624, 1625, 1626, 1633, 1634, 1636, 1648, 1655, 1664, 1665, 1669, 1672, 1680, 1683, 1686, 1692, 1699, 1716, 1731, 1746, 1800, 1816, 1832, 1833, 1848, 1851, 1873, 1876, 1884, 1892, 1899, 1913, 1916, 1932, 1936, 1944, 1953, 1966, 1967, 1977, 1982, 1983, 1984, 1985, 1986, 1990, 1991, 1995, 1996, 1999, 2111, 2112, 2113, 2126, 2128, 2129, 2167, 2178, 2179, 2185, 2201, 2205, 2206, 2208, 2210, 2224, 2225, 2227, 2228, 2233, 2241, 2242, 2243, 2244, 2249, 2261, 2262, 2269, 2274, 2278, 2292, 2312, 2399, 2409, 2427, 2441, 2442, 2443, 2445, 2448, 2450, 2455, 2463, 2477, 2479, 2484, 2487, 2489, 2493, 2496, 2500, 2502, 2512, 2520, 2523, 2525, 2529, 2530, 2538, 2543, 2544, 2545, 2548, 2553, 2554, 2555, 2566, 2569, 2570, 2573, 2575, 2579, 2580, 2588, 2590, 2594, 2598, 2603, 2607, 2614, 2618, 2620, 2637, 2638, 2643, 2651, 2657, 2673, 2761, 2765, 2775, 2783, 2788, 2793, 2797, 2822, 2826, 2835, 2841, 2851, 2852, 2872, 2873, 2877, 2880, 2883, 2885, 2886, 2898, 2902, 2911, 2914, 2918, 2920, 2922, 2926, 2930, 2935, 2936, 2937, 2939, 2942, 2948, 2949, 2954, 2956, 2959, 2960, 2964, 2965, 2968, 2979, 2989, 2992, 2997, 3001, 3012, 3014, 3035, 3043, 3049, 3051, 3054, 3058, 3107, 3121, 3122, 3123, 3129, 3133, 3134, 3136, 3137, 3144, 3145, 3146, 3147, 3149, 3152, 3167, 3170, 3180, 3191, 3209, 3211, 3236, 3239, 3241, 3242, 3249, 3251, 3268, 3285, 3289, 3304, 3305, 3395, 3418, 3422, 3426, 3445, 3463, 3464, 3466, 3473, 3492, 3493, 3501, 3521, 3575, 3576, 3588, 3603, 3604, 3607, 3616, 3631, 3637, 3687, 3702, 3704, 3751, 3763, 3783, 3788, 3789, 3797, 3832, 3844, 3849, 3854, 3855, 3881, 3889, 3895, 3933, 3949, 3955, 3959, 3965, 3971, 3974, 3975, 3979, 3980, 4045, 4046, 4049, 4057, 4061, 4102, 4115, 4123, 4132, 4136, 4155, 4157, 4158, 4181, 4183, 4194, 4205, 4206, 4207, 4208, 4209, 4210, 4227, 4233, 4252, 4297, 4298, 4313, 4417, 4421, 4422, 4423, 4428, 4430, 4431, 4432, 4438, 4444, 4445, 4446, 4448, 4453, 4460, 4466, 4471, 4476, 4480, 4514, 4532, 4545, 4550, 4552, 4553, 4562, 4610, 4611, 4612, 4613, 4614, 4619, 4623, 4630, 4638, 4640, 4657, 4667, 4675, 4676, 4677, 4683, 4693, 4714, 4725, 4740, 4741, 4746, 4749, 4765, 4767, 4778, 4891, 4941, 4987, 4991, 4995, 4997, 4998, 5012, 5017, 5020, 5022, 5023, 5027, 5028, 5032, 5083, 5084, 5085, 5098, 5100, 5106, 5108, 5127, 5131, 5134, 5141, 5150, 5169, 5173, 5179, 5184, 5191, 5199, 5214, 5216, 5220, 5225, 5228, 5233, 5236, 5241, 5245, 5250, 5255, 5257, 5278, 5280, 5281, 5356, 5362, 5363, 5366, 5367, 5369, 5370, 5438, 5442, 5449, 5450, 5458, 5460, 5483, 5492, 5495, 5500, 5518, 5527, 5528, 5530, 5533, 5535, 5574, 5577, 5583, 5596, 5598, 5599, 5605, 5610, 5612, 5618, 5623, 5624, 5648, 5650, 5651, 5654, 5666, 5667, 5671, 5688, 5693, 5698, 5702, 5736, 5754, 5767, 5772, 5774, 5775, 5784, 5832, 5833, 5834, 5835, 5836, 5841, 5845, 5852, 5860, 5862, 5879, 5889, 5897, 5898, 5899, 5905, 5915, 5936, 5947, 5962, 5963, 5968, 5971, 5987, 5989, 6000, 6113, 6163, 6192, 6195, 6211, 6215, 6226, 6230, 6232, 6234, 6297, 6298, 6310, 6312, 6313, 6315, 6317, 6319, 6358, 6383, 6384, 6394, 6397, 6398, 6404, 6419, 6445, 6448, 6477, 6478, 6504, 6514, 6547, 6548, 6549, 6557, 6567, 6607, 6608, 6610, 6611, 6612, 6627, 6628, 6636, 6648, 6650, 6671, 6673, 6674, 6676, 6678, 6691, 6699, 6724, 6728, 6735, 6773, 6779, 6796, 6799, 6823, 6843, 6857, 6866, 6875, 6881, 6891, 6894, 6901, 6913, 6930, 6939, 6940, 6942, 6943, 6966, 6967, 6994, 7002, 7005, 7009, 7016, 7020, 7022, 7026, 7028, 7031, 7035, 7038, 7045, 7064, 7151]

In [23]:
size_exist_df = df.loc[size_exist]
price_exist_df = df.loc[price_exist]
shipment_exist_df = df.loc[shipment_exist]

In [24]:
size_exist_df.head()

,ProductNum,ID,Date,Grade,Text,Option,Size,FilteredText,Rate,FilteredLen
1,3569992365,newr*****,2023.03.27,5,옷에 비침도 크게 없고 사이즈도 넉넉해서 좋습니다. 이번 여름까지 무난하게 잘 입을...,"XX-Large, 6, White 6-pack",비공개,"['비침도', '크게', '없고', '사이즈', '넉넉해서', '좋습니다', '여름...",2,10
17,3569992365,oks1*************,2022.11.14,4,미국에서 반팔티사면 딱 느껴지는 그 거친 질감입니다. 여름에 입기에 좋은 시원한 느...,0,0,"['미국', '사면', '느껴지는', '거친', '질감', '입니다', '여름', ...",3,28
18,3569992365,egsi***,2022.07.11,5,110 사이즈를 입어서 2xl 를 구입했는데 한국 사이즈와 비교시 2사이즈 작게 구...,0,0,"['사이즈', '입어서', '구입', '했는데', '한국', '사이즈', '비교',...",0,22
20,3569992365,코랄라니,2022.04.19,5,남자친구 주려고 샀어요 사이즈큼직하고 면이 빳빳하지 않아서 좋아요 잘 입어질것 같아...,0,0,"['남자친구', '려고', '샀어요', '사이즈', '큼직하고', '빳빳하', '않...",5,18
22,3569992365,쁘뜨리,2022.01.03,5,가성비좋은 상품이네요.173 .62키로 L사이즈 세탁하니 좀줄었지만 잘맞습니다.,0,0,"['가성', '좋은', '상품', '이네', '키로', '사이즈', '세탁', '하...",3,10


In [ ]:
size_exist_df.tail()

In [ ]:
#  사이즈가 언급된 리뷰에서 평점과 텍스트리뷰 긍부정 상관관계 
#'Grade', 'Rate' 컬럼 간 피어슨 상관계수 계산
corr = size_exist_df[['Grade', 'Rate']].corr()
print(corr)

In [27]:
size_exist_df["Rate"].mean()

1.1458868894601542

In [ ]:
price_exist_df.tail()

In [ ]:
#  가격이 언급된 리뷰에서 평점과 텍스트리뷰 긍부정 상관관계 
#'Grade', 'Rate' 컬럼 간 피어슨 상관계수 계산
corr = price_exist_df[['Grade', 'Rate']].corr()
print(corr)

In [29]:
price_exist_df["Rate"].mean()

2.074418604651163

In [ ]:
#  배송이 언급된 리뷰에서 평점과 텍스트리뷰 긍부정 상관관계 
#'Grade', 'Rate' 컬럼 간 피어슨 상관계수 계산
corr = shipment_exist_df[['Grade', 'Rate']].corr()
print(corr)

In [30]:
shipment_exist_df["Rate"].mean()

2.34733893557423